In [4]:
import requests
import time
import datetime
from datetime import datetime as dt
import numpy as np
import sys
import pandas as pd
import sys

In [48]:
# 원하는 날짜-1
# 27일 ~ 28일 -> 28일 ~ 29일

cra = '20200526'
crb = '20200527'
cse = "http://203.253.128.161:7579/Mobius/"
ae = 'ubicomp_lora/'

In [49]:
# 모비우스 저장 시 기준, 한국 시 기준을 맞추기 위함

time_dif = 'T143000'
cra_ = str(int(cra) - 1)
query = f'?rcn=4&cra={cra_+time_dif}&crb={crb+time_dif}'

In [50]:
payload = {}
headers = {
  'Accept': 'application/json',
  'X-M2M-RI': '12345',
  'X-M2M-Origin': 'SOrigin'
}

In [51]:
data = []

df = pd.DataFrame(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])
all_df = pd.DataFrame(columns=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

In [52]:
start = time.time()

In [68]:
from datetime import datetime as dt

cra_d = dt.strptime(cra[0:8], '%Y%m%d')
crb_d = dt.strptime(crb[0:8], '%Y%m%d') + datetime.timedelta(days=1)

In [69]:
d_dif = crb_d - cra_d
dif_min = d_dif.total_seconds() // 60
DATA_LEN = int(dif_min // 30) 
DATA_LEN
#dif_min/60/24

96

In [119]:
# 행 -> 시간 정보 (정상 데이터 개수) crb - cra
# 열 -> 정보 종류 (?) (인덱스) 13
# 테스트용이므로 행 한개만 생성
df = pd.DataFrame('NaN', index=range(DATA_LEN), columns=range(13))

In [79]:
k = 1
cnt = str(k)
url = cse+ae+cnt+query
r = requests.get(url, headers=headers, data=payload)

if r.status_code != 200:
    sys.exit(0)

cin = r.json()['m2m:rsp']['m2m:cin']
cin_df = pd.DataFrame(cin)['con']
cin_split = pd.DataFrame(cin_df.str.replace('\r', '').str.split(',').tolist())

In [80]:
d_str = cin_split[3]+' '+cin_split[4]
d_format = '%Y/%m/%d %H:%M:%S'

In [81]:
for i in range(0, len(cin_split)):
    crd_d = dt.strptime(d_str[i], d_format)
    cin_split.loc[i,13] = int(((crd_d - cra_d).total_seconds()//60)//30)

In [110]:
cin_split = cin_split.astype({13:'int'})
cin_split.sort_values(by=13, inplace=True)
cin_split

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
95,1,418,0,2020/05/26,00:00:52,36.7707,126.9294,37,38,66,86,13.55,3.5,0
94,1,419,1,2020/05/26,00:30:52,36.7707,126.9294,37,39,73,93,13.55,3.5,1
93,1,420,2,2020/05/26,01:00:52,36.7707,126.9294,37,34,58,75,13.55,3.5,2
92,1,421,3,2020/05/26,01:30:52,36.7707,126.9294,37,37,64,84,13.23,3.5,3
91,1,422,4,2020/05/26,02:00:52,36.7707,126.9294,37,30,54,68,13.23,3.5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,1,509,43,2020/05/27,21:30:49,36.7707,126.9294,37,28,44,53,15.16,3.5,91
1,1,510,44,2020/05/27,22:00:49,36.7707,126.9294,37,28,41,48,14.52,3.5,92
28,1,511,45,2020/05/27,22:30:49,36.7707,126.9294,37,29,46,53,13.87,3.5,93
26,1,512,46,2020/05/27,23:00:49,36.7707,126.9294,37,34,52,67,13.87,3.5,94


numpy 로 안바꿔서 넣으면 행 순서가 뒤죽박죽 됨

In [112]:
cin = cin_split.to_numpy()
cin

array([['1', '418', '0', ..., '13.55', '3.5', 0],
       ['1', '419', '1', ..., '13.55', '3.5', 1],
       ['1', '420', '2', ..., '13.55', '3.5', 2],
       ...,
       ['1', '511', '45', ..., '13.87', '3.5', 93],
       ['1', '512', '46', ..., '13.87', '3.5', 94],
       ['1', '513', '47', ..., '12.90', '3.5', 95]], dtype=object)

In [121]:
for i in range(len(cin)):
    df.loc[cin[i,13]] = cin[i,:13]

In [124]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,418,0,2020/05/26,00:00:52,36.7707,126.9294,37,38,66,86,13.55,3.5
1,1,419,1,2020/05/26,00:30:52,36.7707,126.9294,37,39,73,93,13.55,3.5
2,1,420,2,2020/05/26,01:00:52,36.7707,126.9294,37,34,58,75,13.55,3.5
3,1,421,3,2020/05/26,01:30:52,36.7707,126.9294,37,37,64,84,13.23,3.5
4,1,422,4,2020/05/26,02:00:52,36.7707,126.9294,37,30,54,68,13.23,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1,509,43,2020/05/27,21:30:49,36.7707,126.9294,37,28,44,53,15.16,3.5
92,1,510,44,2020/05/27,22:00:49,36.7707,126.9294,37,28,41,48,14.52,3.5
93,1,511,45,2020/05/27,22:30:49,36.7707,126.9294,37,29,46,53,13.87,3.5
94,1,512,46,2020/05/27,23:00:49,36.7707,126.9294,37,34,52,67,13.87,3.5
